### Load imports and run the no LID case:

In [22]:
%reset
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import runswmm
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys
def runSWMM(runsCollection,runParamList):
    dateTime = datetime.now()
    (peak,volume,lidDict) = runswmm(runParamList)
    run = { "peak": peak, "volume": volume, "lidDict": lidDict, "runParamList": runParamList,
            "dateTime": dateTime}
    doc_id = runsCollection.insert_one(run).inserted_id
    return doc_id
    # print "Successfully stored %s" % doc_id
#client = MongoClient('mongodb://server.mcgarity.info:27017/')
client = MongoClient()
db = client.swmmwiseDebug
runsToday = db.y16m01d22
f = open('runlist_start.yaml','r')   # Read the no LID changable parameters from file
noLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
noLidDocId = runSWMM(runsToday,noLidList)  # run SWMM on the no LID model for baseline
cursor = runsToday.find_one({'_id':noLidDocId})
noLidPeak = cursor['peak']
noLidVolume = cursor['volume']
print "Stored record %s for no LID parameters, peak = %s, volume = %s" % (noLidDocId,noLidPeak,noLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Stored record 56a3a3b0abd22f9c8ac220c0 for no LID parameters, peak = 21.52, volume = 15.563


### Change ALL subcatchment LID numbers:

In [23]:
numberForAllLid = 1
newRunList=list(noLidList)
for lid in newRunList:
    lid['Number'] = numberForAllLid
newRunDocId = runSWMM(runsToday,newRunList)
cursor = runsToday.find_one({'_id':newRunDocId})
newPeak = cursor['peak']
newVolume = cursor['volume']
peakReduction = noLidPeak - newPeak
volumeReduction = noLidVolume - newVolume
print "Stored record %s for new parameters: peak = %s, volume = %s" % (newRunDocId,newPeak,newVolume)
print "peakReduction = %s, volumeReduction = %s" % (peakReduction,volumeReduction)

Stored record 56a3a3c5abd22f9c8ac220c1 for new parameters: peak = 21.5, volume = 15.109
peakReduction = 0.02, volumeReduction = 0.454


In [5]:
runs = db.y16m01d22
cursor = runs.find()
last = cursor.count()-1
cursor[last]

{u'_id': ObjectId('56a2d17fabd22f9c8ac220ba'),
 u'dateTime': datetime.datetime(2016, 1, 22, 20, 3, 47, 662000),
 u'lidDict': None,
 u'peak': 21.52,
 u'runParamList': [{u'Area': 741,
   u'FromImp': 100,
   u'LID': u'wakefield_BR_RG',
   u'Number': 0,
   u'Subcat': u'S1',
   u'Width': 0}],
 u'volume': 15.563}

In [21]:
runs = db.y16m01d22
cursor = runs.find_one({'_id':noLidDocId})
print cursor
print ''
cursor = runs.find_one({'_id':newRunDocId})
print cursor

{u'lidDict': None, u'dateTime': datetime.datetime(2016, 1, 22, 20, 44, 14, 282000), u'volume': 15.563, u'peak': 21.52, u'runParamList': [{u'LID': u'wakefield_BR_RG', u'Subcat': u'S1', u'Area': 741, u'Number': 0, u'Width': 0, u'FromImp': 100}], u'_id': ObjectId('56a2dafaabd22f9c8ac220bb')}

{u'lidDict': {u'S1 wakefield_BR_RG': {u'Infil Loss': 966.44, u'Total Inflow': 5250.33, u'Evap Loss': 21.2, u'Surface Outflow': 4262.7, u'Continuity Error': -0.0, u'Initial Storage': 0.3, u'Final Storage': 0.5, u'Drain Outflow': 0.0}}, u'dateTime': datetime.datetime(2016, 1, 22, 21, 18, 59, 274000), u'volume': 15.109, u'peak': 21.5, u'runParamList': [{u'LID': u'wakefield_BR_RG', u'Subcat': u'S1', u'Area': 741, u'Number': 1, u'Width': 0, u'FromImp': 100}], u'_id': ObjectId('56a2e31fabd22f9c8ac220bf')}
